### The Challenge

1. **Activity completion rate by theme**

    Write a SQL query that shows the completion rate for each theme. Completion rate = (completed activities / total activity attempts) * 100.

2. **Premium vs Free Content engagement**

    Write a SQL query that compares average session duration between free and premium activities, showing the count of sessions and average duration for each price tier.
3. **User subscription impact**


   Write a SQL query that shows, for each platform, the average number of activities completed per user in the 7 days before vs 7 days after their subscription date. Only include users who subscribed in January 2025.



In [0]:
%sql

-- Activity completion rate by theme
select 
  activities.theme,
  (sum(CASE WHEN activity_events.completed = TRUE THEN 1 else 0 END) /  count(distinct activity_events.event_id)) as completion_rate
from
  lingokids.analytics_engineer.activity_events
left join lingokids.analytics_engineer.activities
  on activity_events.activity_id = activities.activity_id
where event_type = 'activity_exit'
group by activities.theme

In [0]:
%sql

--2. **Premium vs Free Content engagement**
-- Write a SQL query that compares average session duration between free and premium activities, showing the count of sessions and average duration for each price tier.

select
  price_tier,
  count(distinct event_id) count_of_sessions,
  avg(duration_seconds) avg_duration_seconds
from
  lingokids.analytics_engineer.activity_events
left join lingokids.analytics_engineer.activities
  on activity_events.activity_id = activities.activity_id
where event_type = "activity_exit"
group by price_tier

In [0]:
%sql

-- 3. **User subscription impact**


--   Write a SQL query that shows, for each platform, the average number of activities completed per user in the 7 days before vs 7 days after their subscription date. Only include users who subscribed in January 2025.

select 
  summary_plays_user.platform,
  avg(before_count) as avg_activities_before_per_user,
  avg(after_count) as avg_activities_after_per_user
from (
  select subscriptions.platform,
    subscriptions.user_id,
    sum(case 
              when subscriptions.subscription_date <= activity_events.created_at 
                and subscriptions.subscription_date >= activity_events.created_at - INTERVAL 7 DAYS then 1
              else 0 end) as after_count,
    sum(case             
              when subscriptions.subscription_date > activity_events.created_at 
                and subscriptions.subscription_date < activity_events.created_at + INTERVAL 7 DAYS then 1
            else 0 end) as before_count,
    count(distinct activity_events.event_id)
  from
    lingokids.analytics_engineer.subscriptions
  inner join
    lingokids.analytics_engineer.activity_events
    on subscriptions.user_id = activity_events.user_id
  where month(subscriptions.subscription_date) = 1
    and year(subscriptions.subscription_date) = 2025 -- filter only people who subscribed in january 2025
    and activity_events.event_type = "activity_exit" -- to avoid the duplicity of entering and exit events
    and case 
              when subscriptions.subscription_date <= activity_events.created_at 
                and subscriptions.subscription_date >= activity_events.created_at - INTERVAL 7 DAYS then 'after'
              when subscriptions.subscription_date > activity_events.created_at 
                and subscriptions.subscription_date < activity_events.created_at + INTERVAL 7 DAYS then 'before'
            else 'not applicable' end != 'not applicable' -- we filter out the cases where the subscription date is not in the 7 days before or after the activity event
  group by  
      subscriptions.platform, 
      subscriptions.user_id
      ) summary_plays_user

group by 
  summary_plays_user.platform


In [0]:
%sql

select subscriptions.platform,
  subscriptions.user_id,
  subscriptions.subscription_date,
  activity_events.created_at,
  case 
            when subscriptions.subscription_date <= activity_events.created_at 
              and subscriptions.subscription_date >= activity_events.created_at - INTERVAL 7 DAYS then 'after'
               
            when subscriptions.subscription_date > activity_events.created_at 
              and subscriptions.subscription_date < activity_events.created_at + INTERVAL 7 DAYS then 'before'
          else 'not applicable' end as whatever,
  

  sum(case 
            when subscriptions.subscription_date <= activity_events.created_at 
              and subscriptions.subscription_date >= activity_events.created_at - INTERVAL 7 DAYS then 1
            else 0 end) as after_count,
  sum(case             
            when subscriptions.subscription_date > activity_events.created_at 
              and subscriptions.subscription_date < activity_events.created_at + INTERVAL 7 DAYS then 1
          else 0 end) as before_count,
  count(distinct activity_events.event_id)
from
  lingokids.analytics_engineer.subscriptions
inner join
  lingokids.analytics_engineer.activity_events
  on subscriptions.user_id = activity_events.user_id
where month(subscriptions.subscription_date) = 1
  and year(subscriptions.subscription_date) = 2025
  and activity_events.event_type = "activity_exit" -- to avoid the duplicity of entering and exit events
  and case 
            when subscriptions.subscription_date <= activity_events.created_at 
              and subscriptions.subscription_date >= activity_events.created_at - INTERVAL 7 DAYS then 'after'
            when subscriptions.subscription_date > activity_events.created_at 
              and subscriptions.subscription_date < activity_events.created_at + INTERVAL 7 DAYS then 'before'
          else 'not applicable' end != 'not applicable' -- we filter out the cases where the subscription date is not in the 7 days before or after the activity event
group by  
    subscriptions.platform, 
    subscriptions.user_id,
      case 
            when subscriptions.subscription_date <= activity_events.created_at 
              and subscriptions.subscription_date >= activity_events.created_at - INTERVAL 7 DAYS then 'after'
               
            when subscriptions.subscription_date > activity_events.created_at 
              and subscriptions.subscription_date < activity_events.created_at + INTERVAL 7 DAYS then 'before'
          else 'not applicable' end,
    subscriptions.subscription_date,
    activity_events.created_at    
